# 4E - 14 DAYS

In [1]:
%load_ext autoreload
%autoreload 2
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt 
import seaborn as sns
import json
from tensorflow.keras.models import load_model
from scipy.stats import zscore
from sklearn.preprocessing import MinMaxScaler

In [2]:
## Change directory to where your scripts and data file is placed ### 
BASE_PATH = "./private/thesisFinalFiles/"
os.chdir(BASE_PATH)
print(os.path.abspath('')) 

from spacetimemodel import spacetimemodel
from preprocessing_exp2 import *

/data/private/thesisFinalFiles


In [3]:
data = pd.read_csv('vietnam_dataMatrix_modelling2.csv', header=0)
idx_dic = {}
for col in data.columns:
    idx_dic[col] = data.columns.get_loc(col)
print(idx_dic)


{'OID_': 0, 'area': 1, 'landslide': 2, 'su_ID': 3, 'eventDate': 4, 'inventory': 5, 'reliefMax': 6, 'reliefSD': 7, 'planCurMax': 8, 'planCurvSD': 9, 'profCurMax': 10, 'profCurSD': 11, 'slopeMax': 12, 'slopeSD': 13, 'faultDiMax': 14, 'faultDiSD': 15, 'litho': 16, 'landcover': 17, 'F1_max': 18, 'F1_sum': 19, 'F2_max': 20, 'F2_sum': 21, 'F3_max': 22, 'F3_sum': 23, 'F4_max': 24, 'F4_sum': 25, 'F5_max': 26, 'F5_sum': 27, 'F6_max': 28, 'F6_sum': 29, 'F7_max': 30, 'F7_sum': 31, 'F8_max': 32, 'F8_sum': 33, 'F9_max': 34, 'F9_sum': 35, 'F10_max': 36, 'F10_sum': 37, 'F11_max': 38, 'F11_sum': 39, 'F12_max': 40, 'F12_sum': 41, 'F13_max': 42, 'F13_sum': 43, 'F14_max': 44, 'F14_sum': 45, 'F15_max': 46, 'F15_sum': 47, 'F16_max': 48, 'F16_sum': 49, 'F17_max': 50, 'F17_sum': 51, 'F18_max': 52, 'F18_sum': 53, 'F19_max': 54, 'F19_sum': 55, 'F20_max': 56, 'F20_sum': 57, 'F21_max': 58, 'F21_sum': 59, 'F22_max': 60, 'F22_sum': 61, 'F23_max': 62, 'F23_sum': 63, 'F24_max': 64, 'F24_sum': 65, 'F25_max': 66, 'F25

# Preparing rainfall data

## difference

In [4]:
# create a new DataFrame to hold the differences
diffMax_df = pd.DataFrame()
diffSum_df = pd.DataFrame()

df_max = data.iloc[:,np.arange(50,78,2)]
df_sum = data.iloc[:,np.arange(51,78,2)]

# iterate over the columns of the original DataFrame
for i in range(1, len(df_max.columns)):
    if i == 1: 
        diffMax_df[f'diff_{1}'] = 0

    # calculate the difference between this column and the previous column
    diff = df_max.iloc[:, i] - df_max.iloc[:, i-1]
    # add the difference as a new column in the diff_df DataFrame
    diffMax_df[f'diff_{i+1}'] = diff

diffMax_df['diff_1'].fillna(value=0,inplace=True)

for i in range(1, len(df_max.columns)):
    if i == 1: 
        diffSum_df[f'diff_{1}'] = 0

    # calculate the difference between this column and the previous column
    diff = df_sum.iloc[:, i] - df_sum.iloc[:, i-1]
    # add the difference as a new column in the diff_df DataFrame
    diffSum_df[f'diff_{i+1}'] = diff

diffSum_df['diff_1'].fillna(value=0,inplace=True)

# # display the resulting DataFrame
diffSum_df.head(10)

diff_1     diff_2     diff_3    diff_4    diff_5     diff_6     diff_7  \
0     0.0   9.710696  16.184493 -3.236900  0.215438 -17.868099   6.540532   
1     0.0   6.144605  10.241009 -2.048202  0.000000 -11.827802   5.151284   
2     0.0  10.557746  17.596243 -3.519248  2.936230 -24.051722   9.780282   
3     0.0   7.531363  11.900925 -4.218611  1.940526 -12.691337   4.967809   
4     0.0  12.447235  19.605534 -7.585746  3.151703 -19.605534   7.085655   
5     0.0   9.887270  16.478782 -3.295755  3.295755 -23.070295  10.147362   
6     0.0   6.512400  10.853999 -2.170801  0.174945 -13.132517   6.057903   
7     0.0   6.507313   3.904389 -2.602926  1.301463  -6.507314   2.755542   
8     0.0   6.983759  11.639598 -2.327920  1.075618 -13.967517   5.718315   
9     0.0   4.594466   7.657443 -1.531489  0.000000  -9.188932   4.311626   

     diff_8     diff_9   diff_10    diff_11    diff_12   diff_13   diff_14  
0  3.848720 -15.394880  0.979931  10.601730   9.434524 -5.588925  2.961902  
1  2.553631 -10.214524  0.255634   6.639711   6.098299 -2.396875  0.772670  
2  4.896277 -15.921331 -1.927152  11.025055   9.234516 -2.575861 -0.362949  
3  2.395039  -8.927871 -1.367964   6.119515   6.849701 -3.851039  2.069589  
4  4.749598 -15.098847 -2.233124  10.065898  10.840843 -5.675763  3.378005  
5  1.360612 -10.574092 -2.114819   8.459274  11.177274 -8.156763  2.718921  
6  2.837765 -11.133694  0.000000   6.849877   6.648347 -1.687277  0.000000  
7  1.786155  -5.358467 -0.893078   3.572312   2.769399  0.000000  0.000000  
8  4.199885 -13.321738  0.000000   6.587186   8.297678 -1.860607  0.000000  
9  1.947706  -7.790820  0.000000   4.869262   4.632146 -1.187675  0.000000

In [5]:
df_cumSumRainfall, df_sumRainfall = cumRainfall(data, 51, 77, 'sum')
df_cumMaxRainfall, df_maxRainfall = cumRainfall(data, 50, 77, 'max')
# df_maxRainfall.head(10)

In [6]:
df_sumRainfall.head(10)

F17_sum    F18_sum    F19_sum    F20_sum    F21_sum   F22_sum    F23_sum  \
0        0   9.710696  25.895189  22.658289  22.873727  5.005628  11.546160   
1        0   6.144605  16.385614  14.337412  14.337412  2.509609   7.660893   
2        0  10.557746  28.153989  24.634741  27.570970  3.519249  13.299530   
3        0   7.531363  19.432288  15.213677  17.154203  4.462866   9.430675   
4        0  12.447235  32.052768  24.467023  27.618726  8.013192  15.098847   
5        0   9.887270  26.366052  23.070297  26.366052  3.295756  13.443118   
6        0   6.512400  17.366399  15.195598  15.370543  2.238026   8.295929   
7        0   6.507313  10.411702   7.808776   9.110239  2.602925   5.358467   
8        0   6.983759  18.623357  16.295437  17.371055  3.403538   9.121853   
9        0   4.594466  12.251909  10.720420  10.720420  1.531489   5.843114   

     F24_sum   F25_sum   F26_sum    F27_sum    F28_sum    F29_sum    F30_sum  
0  15.394880  0.000000  0.979931  11.581661  21.016185  15.427260  18.389162  
1  10.214524  0.000000  0.255634   6.895345  12.993644  10.596769  11.369439  
2  18.195807  2.274476  0.347324  11.372379  20.606895  18.031034  17.668085  
3  11.825714  2.897843  1.529879   7.649394  14.499095  10.648056  12.717645  
4  19.848445  4.749598  2.516475  12.582372  23.423215  17.747452  21.125457  
5  14.803730  4.229637  2.114819  10.574093  21.751367  13.594604  16.313526  
6  11.133694  0.000000  0.000000   6.849877  13.498224  11.810947  11.810947  
7   7.144622  1.786156  0.893078   4.465389   7.234789   7.234789   7.234789  
8  13.321738  0.000000  0.000000   6.587186  14.884865  13.024258  13.024258  
9   7.790820  0.000000  0.000000   4.869262   9.501409   8.313733   8.313733

In [7]:
scaler = MinMaxScaler()

df_sumDiffScaled= scaler.fit_transform(diffSum_df)
df_sumDiffScaled = pd.DataFrame(df_sumDiffScaled, columns=diffSum_df.columns)
df_sumDiffScaled.head(2)

diff_1    diff_2    diff_3    diff_4    diff_5    diff_6    diff_7  \
0     0.0  0.732172  0.773510  0.348107  0.498392  0.158549  0.596939   
1     0.0  0.637948  0.616015  0.379334  0.493465  0.283835  0.559144   

     diff_8    diff_9   diff_10   diff_11   diff_12   diff_13   diff_14  
0  0.432004  0.237318  0.416769  0.866596  0.685095  0.171108  0.533706  
1  0.398977  0.359422  0.379240  0.712965  0.621861  0.250050  0.490652

In [13]:
rainfall_array = rainfallMatrix_4(df_maxRainfall.apply(zscore), df_sumRainfall.apply(zscore), df_cumSumRainfall.apply(zscore), df_sumDiffScaled.apply(zscore))

In [14]:
rainfall_array.shape

(2759, 14, 4)

# Processing environmental data

In [15]:
# normalizing continuous static environmental data 
df_staticOnly = data[['litho','landcover',
                    'reliefSD', 'slopeSD', 'cosAspSD', 'sinAspSD', 'faultDiSD',
                     'reliefME', 'plancurME', 'profcurvME', 'slopeME','fautltDiME','sinAspME', 'cosAspME' ]]

catCols = ['landcover','litho']
df_staticOnlyNorm = normCols(df_staticOnly, catCols)
# df_staticOnlyNorm.head(10)

litho          object
landcover      object
reliefSD      float64
slopeSD       float64
cosAspSD      float64
sinAspSD      float64
faultDiSD     float64
reliefME      float64
plancurME     float64
profcurvME    float64
slopeME       float64
fautltDiME    float64
sinAspME      float64
cosAspME      float64
dtype: object 

Index(['reliefSD', 'slopeSD', 'cosAspSD', 'sinAspSD', 'faultDiSD', 'reliefME',
       'plancurME', 'profcurvME', 'slopeME', 'fautltDiME', 'sinAspME',
       'cosAspME'],
      dtype='object')


In [16]:
# one hot encoding the categorical variables in the df 
in_array_env, outDf = catPrep(df_staticOnlyNorm, ['litho','landcover'])

outDf.columns
len(outDf.columns)
env_array = outDf.to_numpy()
env_array.shape

landslide_array = np.array(data['landslide'], dtype=int)
# print(landslide_array)


You have one-hot encoded  2  categorical variables. These are: 

litho 

landcover 



# Space-Time Modelling

In [18]:
#split data first into train and test 
idx=np.arange(0,2759)
idx_tr, idx_ts = train_test_split(idx, test_size=0.25,random_state=42)
idx_tr.sort()
idx_ts.sort()

# Hyperparameterization

## Hyperband optimization

In [21]:
from optimize_model import * 
import keras_tuner as kt

# params= json.load(open('Exp4E_bay_params.json','r'))
# print(params)

X_train = [env_array[idx_tr],rainfall_array[idx_tr]]
Y_train = landslide_array[idx_tr]

obj = optimize_model(params['modelparam'])
# objfunc = kt.Objective('val_auc', direction='max')
objfunc = kt.Objective('val_auc', direction='max')
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_auc', patience=10, restore_best_weights=True)
tuner2 = kt.Hyperband(obj, objective=objfunc, 
                       max_epochs=150, factor=3, 
                      project_name='msc-tuning-hyperband-4e-14days',
                      overwrite=True, seed=10)
tuner2.search(X_train, Y_train, epochs=200, validation_split=0.2, callbacks=[stop_early]) #tf.keras.callbacks.TensorBoard("/tmp/tb_logs")
best_hps = tuner2.get_best_hyperparameters(num_trials=2)[0]

Trial 254 Complete [00h 12m 22s]
val_auc: 0.6960534453392029

Best val_auc So Far: 0.7346550822257996
Total elapsed time: 03h 49m 10s
INFO:tensorflow:Oracle triggered exit


INFO:tensorflow:Oracle triggered exit


In [22]:
best_hps.values

{'GRU Layer': 48,
 'Env Layer': 16,
 'Combined Layer': 64,
 'Env depth': 5,
 'Comb depth': 4,
 'learning_rate': 1e-06,
 'GRU activation': 'tanh',
 'Env activation': 'tanh',
 'Comb activation': 'tanh',
 'gru dropout ratio': 0.15,
 'env dropout ratio': 0.1,
 'comb dropout ratio': 0.05,
 'batch_size': 14,
 'tuner/epochs': 150,
 'tuner/initial_epoch': 50,
 'tuner/bracket': 4,
 'tuner/round': 4,
 'tuner/trial_id': '0142'}

## Bayesian optimization

In [20]:
from optimize_model import * 
import keras_tuner as kt

# params= json.load(open('Exp4E_params.json','r'))

X_train = [env_array[idx_tr],rainfall_array[idx_tr]]
Y_train = landslide_array[idx_tr]
objfunc = kt.Objective('val_auc', direction='max')
obj = optimize_model(params['modelparam'])

# note that in older versions, it's obj.build but as of 4/4/23, if build is there, batch size somehow ain't hyperparameterized.
tuner3 = kt.BayesianOptimization(obj, objective=objfunc, 
                      project_name='msc-tuning-4E-14days', max_trials=50, max_retries_per_trial=5,
                      overwrite=True, seed=10)
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_auc', patience=10, restore_best_weights=True)
tuner3.search(X_train, Y_train, epochs=200, validation_split=0.2, callbacks=[stop_early])
best_hps3 = tuner3.get_best_hyperparameters(num_trials=2)[0]

Trial 50 Complete [00h 00m 49s]
val_auc: 0.655864953994751

Best val_auc So Far: 0.7642480731010437
Total elapsed time: 05h 08m 41s
INFO:tensorflow:Oracle triggered exit


INFO:tensorflow:Oracle triggered exit


In [23]:
best_hps3.values


{'GRU Layer': 64,
 'Env Layer': 16,
 'Combined Layer': 16,
 'Env depth': 2,
 'Comb depth': 1,
 'learning_rate': 1e-05,
 'GRU activation': 'relu',
 'Env activation': 'tanh',
 'Comb activation': 'swish',
 'gru dropout ratio': 0.05,
 'env dropout ratio': 0.1,
 'comb dropout ratio': 0.15,
 'batch_size': 21}